# Single Domain Experiments

In [ ]:
#we first connect to my google drive in order to collect all the data
#we will have three domains (restaurant, hotels and electronics)
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#function to extract the aspects from the sentence and calculate the f1 for each sentence
#The f1 values will be printed on a file in order to compute statistics
def print_f1_on_file(filename,true_aspects,extracted_aspects):
  count = 0
  f1 = 0.0
  f = open(filename,"w+")
  for ea,ta in zip(extracted_aspects,true_aspects):
    if 'B-aspect' in ta:
      f.write(str(f1_score([ta],[ea]))+"\n")
      count += 1
      f1 += f1_score([ta],[ea])
  f.close()
  print(f1)
  print(count)
  print(f1/count)

In [ ]:
#we use the tensorflow version 1.x
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
%tensorflow_version 1.x
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

TensorFlow 1.x selected.


In [ ]:
#first try with restaurant dataset
#In this case we perform a single domain experiment on the restaurant dataset
#If we are interested in other domains, we only need to change the file
data = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/restaurants1.csv", encoding="latin1", sep="\t")
data = data.fillna(method="ffill")
data.tail(10)

,SENTENCE,TOKEN,TAG
71615,4673,",",O
71616,4673,various,O
71617,4673,vegetables,B-aspect
71618,4673,",",O
71619,4673,and,O
71620,4673,rice,B-aspect
71621,4673,and,O
71622,4673,glass,B-aspect
71623,4673,noodles,I-aspect
71624,4673,.,O


In [ ]:
#creation of a set containing the words
#this will be useful in order to create a mappin between the tokens in the sentences and the words
words = set(list(data['TOKEN'].values))
words.add('PADword')
n_words = len(words)
print(n_words)

6924


In [ ]:
#Creation of a set containing all the possible tags
tags = list(set(data["TAG"].values))
n_tags = len(tags)
print(n_tags)

3


In [ ]:
#class for get sentences in the format of tuples [(TOKEN,TAG),(TOKEN,TAG)...]
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["TOKEN"].values.tolist(),s["TAG"].values.tolist())]
        self.grouped = self.data.groupby("SENTENCE").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
#we get all the sentences and we print the longest sentence
getter = SentenceGetter(data)
sentences = getter.sentences
print("# of sentences: ",len(sentences))
largest_sen = max(len(sen) for sen in sentences)
print('Biggest sentence has {} words'.format(largest_sen))

# of sentences:  4674
Biggest sentence has 76 words


In [ ]:
#we print the first sentence in the above format
print(sentences[0])

[('Judging', 'O'), ('from', 'O'), ('previous', 'O'), ('posts', 'O'), ('this', 'O'), ('used', 'O'), ('to', 'O'), ('be', 'O'), ('a', 'O'), ('good', 'O'), ('place', 'B-aspect'), (',', 'O'), ('but', 'O'), ('not', 'O'), ('any', 'O'), ('longer', 'O'), ('.', 'O')]


In [ ]:
#we create the X dataset containing only the padded sentences' tokens
max_len = 75
X = [[w[0]for w in s] for s in sentences]
new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X.append(new_seq)

print(new_X[0])

['Judging', 'from', 'previous', 'posts', 'this', 'used', 'to', 'be', 'a', 'good', 'place', ',', 'but', 'not', 'any', 'longer', '.', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword']


In [ ]:
#we convert the tags into indeces
#we also create the padded y
from keras.preprocessing.sequence import pad_sequences
tags2index = {t:i for i,t in enumerate(tags)}
y = [[tags2index[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tags2index["O"])
print("Mapping between indeces and tags:",tags2index)

Mapping between indeces and tags: {'I-aspect': 0, 'B-aspect': 1, 'O': 2}


In [ ]:
#We instal seqeval for evaluating the perfomances of the model
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7423 sha256=dde500a0963ae1458b7b76922dd920dec08b618d46ea09423a97fe370d37e3f5
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [ ]:
#We import the needed libraries
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

In [ ]:
#We define the ELMo embedding function that will be used by our embedding layer in the neural network
batch_size = 32
def ElmoEmbedding(x):
    return elmo_model(inputs={"tokens": tf.squeeze(tf.cast(x,    tf.string)),"sequence_len": tf.constant(batch_size*[max_len])
                     },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [ ]:
#Function to convert the predictions into tags using the argmax function
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PADword", "O"))
        out.append(out_i)
    return out

#Function to convert the test indeces into tags   
def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p].replace("PADword", "O"))
        out.append(out_i)
    return out

In [ ]:
#We create a dictionary for mapping indeces into tags
idx2tag = {i: w for w, i in tags2index.items()}
print(idx2tag)

cv = KFold(n_splits=5, random_state=42, shuffle=False)
i = 0

#We iterate over the folds
for train_index, test_index in cv.split(new_X):
    x_pd = pd.Series(new_X)
    y_pd = y

    #We generate the current training set and test set
    X_tr, X_te, y_tr, y_te = list(x_pd[train_index]),list(x_pd[test_index]),(y_pd[train_index]),(y_pd[test_index])
    
    #We download the pretrained elmo model
    sess = tf.Session()
    K.set_session(sess)
    elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())

    #We define our model
    input_text = Input(shape=(max_len,), dtype=tf.string)
    embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
    
    x = Bidirectional(LSTM(units=256, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
    x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
    out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
    model = Model(input_text, out)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    l = len(X_tr)

    #We also divide the training set into training and validation set
    X_tr, X_val = X_tr[:int(l*0.8)], X_tr[-(int(l*0.2)):]
    y_tr, y_val = y_tr[:int(l*0.8)], y_tr[-(int(l*0.2)):]
    

    #We convert training, validation and test size into a multiple of the batch size
    #training set
    l = len(X_tr)
    for i in range(0, 32 - (l%32)):
      new_seq = []
      new_tags = []
      for j in range(max_len):
        new_seq.append("PADword")
        new_tags.append(tags2index['O'])
      y_tr = np.append(y_tr,np.array(new_tags).reshape((1,75)),axis=0)
      X_tr.append(new_seq)

    #validation set
    l = len(X_val)
    for i in range(0, 32 - (l%32)):
      new_seq = []
      new_tags = []
      for j in range(max_len):
        new_seq.append("PADword")
        new_tags.append(tags2index['O'])
      y_val = np.append(y_val,np.array(new_tags).reshape((1,75)),axis = 0)
      X_val.append(new_seq)

    #test set
    l = len(X_te)
    for i in range(0, 32 - (l%32)):
      new_seq = []
      for j in range(max_len):
        new_seq.append("PADword")
      X_te.append(new_seq)

    y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
    y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

    #fitting the model
    history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),batch_size=batch_size, epochs=10, verbose=1)

    #estimating the predictions
    test_pred = model.predict(np.array(X_te), verbose=1)
    pred_labels = pred2label(test_pred)
    test_labels = test2label(y_te)

    #print the current classification report
    print(classification_report(test_labels,pred_labels))

{0: 'I-aspect', 1: 'B-aspect', 2: 'O'}


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Train on 3008 samples, validate on 768 samples
Epoch 1/10
2528/3008 [========================>.....] - ETA: 3:32 - loss: 0.0728 - accuracy: 0.9720

KeyboardInterrupt: ignored

# Cross Domain Experiments

In [ ]:
#we load all the datasets 
data_lap = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/laptops_2014.csv", encoding="latin1", sep="\t")
data_lap = data_lap.fillna(method="ffill")

data_comp = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/Computer.csv", encoding="latin1", sep="\t")
data_comp = data_comp.fillna(method="ffill")

data_router = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/Router.csv", encoding="latin1", sep="\t")
data_router = data_router.fillna(method="ffill")

data_speaker = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/Speaker.csv", encoding="latin1", sep="\t")
data_speaker = data_speaker.fillna(method="ffill")

data_rest = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/restaurants1.csv", encoding="latin1", sep="\t")
data_rest = data_rest.fillna(method="ffill")

data_hotels = pd.read_csv("/content/drive/My Drive/Tesi_ABSA/dataset/hotels.csv", encoding="latin1", sep="\t")
data_hotels = data_hotels.fillna(method="ffill")

In [ ]:
#creation of a set containing the words
#this will be useful in order to create a mappin between the tokens in the sentences and the words
#it is the first step for the creation of a numerical representation of the sentences
#we also add the word PADword for feed the sentences with less than 75 words in order to have all sentences with length 75
words_lap = set(list(data_lap['TOKEN'].values))
words_comp = set(list(data_comp['TOKEN'].values))
words_router = set(list(data_router['TOKEN'].values))
words_speaker = set(list(data_speaker['TOKEN'].values))
words_rest = set(list(data_rest['TOKEN'].values))
words_hotels = set(list(data_hotels['TOKEN'].values))
words = set({})
words = words.union(words_lap,words_comp,words_router,words_speaker,words_rest,words_hotels)
words.add('PADword')
n_words = len(words)
print("# of words in the datasets:",n_words)

# of words in the datasets: 12452


In [ ]:
#for the same reason we create the set containing the unique tags
tags = list(set(data_lap["TAG"].values))
n_tags = len(tags)
print("# of tags in the datasets:",n_tags)

# of tags in the datasets: 3


In [ ]:
#we get all the sentences and we print the longest sentence
getter_lap = SentenceGetter(data_lap)
sentences_lap = getter_lap.sentences
print("# of sentences in the laptop dataset:",len(sentences_lap))

getter_comp = SentenceGetter(data_comp)
sentences_comp = getter_comp.sentences
print("# of sentences in the computers dataset:",len(sentences_comp))

getter_router = SentenceGetter(data_router)
sentences_router = getter_router.sentences
print("# of sentences in the routers dataset:",len(sentences_router))

getter_speaker = SentenceGetter(data_speaker)
sentences_speaker = getter_speaker.sentences
print("# of sentences in the speakers dataset:",len(sentences_speaker))

getter_rest = SentenceGetter(data_rest)
sentences_rest = getter_rest.sentences
print("# of sentences in the restaurants dataset:",len(sentences_rest))

getter_hotels = SentenceGetter(data_hotels)
sentences_hotels = getter_hotels.sentences
print("# of sentences in the hotels dataset:",len(sentences_hotels))

# of sentences in the laptop dataset: 3845
# of sentences in the computers dataset: 531
# of sentences in the routers dataset: 879
# of sentences in the speakers dataset: 689
# of sentences in the restaurants dataset: 4674
# of sentences in the hotels dataset: 266


In [ ]:
#we print the first sentence in the above format
print(sentences_router[0])

[('Affordable', 'O'), ('price', 'B-aspect'), (',', 'O'), ('reliable', 'O'), ('item', 'B-aspect'), ('and', 'O'), ('excellent', 'O'), ('customer', 'B-aspect'), ('service', 'I-aspect'), ('.', 'O')]


In [ ]:
#we create the X dataset containing only the padded sentences' tokens
max_len = 100
print("First padded sentence of each dataset:")
X = [[w[0]for w in s] for s in sentences_lap]
new_X_lap = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_lap.append(new_seq)
print(new_X_lap[0])

X = [[w[0]for w in s] for s in sentences_comp]
new_X_comp = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_comp.append(new_seq)
print(new_X_comp[0])

X = [[w[0]for w in s] for s in sentences_router]
new_X_router = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_router.append(new_seq)
print(new_X_router[0])

X = [[w[0]for w in s] for s in sentences_speaker]
new_X_speaker = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_speaker.append(new_seq)
print(new_X_speaker[0])

X = [[w[0]for w in s] for s in sentences_rest]
new_X_rest = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_rest.append(new_seq)
print(new_X_rest[0])

X = [[w[0]for w in s] for s in sentences_hotels]
new_X_hotels = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X_hotels.append(new_seq)
print(new_X_hotels[0])

First padded sentence of each dataset:
['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADword', 'PADw

In [ ]:
#we also create the padded y 
from keras.preprocessing.sequence import pad_sequences
tags2index = {"O":0,"B-aspect":1,"I-aspect":2}
y_lap = [[tags2index[w[1]] for w in s] for s in sentences_lap]
y_lap = pad_sequences(maxlen=max_len, sequences=y_lap, padding="post", value=tags2index["O"])

y_comp = [[tags2index[w[1]] for w in s] for s in sentences_comp]
y_comp = pad_sequences(maxlen=max_len, sequences=y_comp, padding="post", value=tags2index["O"])

y_router = [[tags2index[w[1]] for w in s] for s in sentences_router]
y_router = pad_sequences(maxlen=max_len, sequences=y_router, padding="post", value=tags2index["O"])

y_speaker = [[tags2index[w[1]] for w in s] for s in sentences_speaker]
y_speaker = pad_sequences(maxlen=max_len, sequences=y_speaker, padding="post", value=tags2index["O"])

y_rest = [[tags2index[w[1]] for w in s] for s in sentences_rest]
y_rest = pad_sequences(maxlen=max_len, sequences=y_rest, padding="post", value=tags2index["O"])

y_hotels = [[tags2index[w[1]] for w in s] for s in sentences_hotels]
y_hotels = pad_sequences(maxlen=max_len, sequences=y_hotels, padding="post", value=tags2index["O"])

In [ ]:
!pip install seqeval

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
#We need to generate our training, validation and test set
#The idea is to train on 5 domains and test on the remaining one
#We split all the six dataset into training and test set
X_tr_lap, X_te_lap, y_tr_lap, y_te_lap = train_test_split(new_X_lap, y_lap, test_size=0.2, random_state=2018)
X_tr_comp, X_te_comp, y_tr_comp, y_te_comp = train_test_split(new_X_comp, y_comp, test_size=0.2, random_state=2018)
X_tr_router, X_te_router, y_tr_router, y_te_router = train_test_split(new_X_router, y_router, test_size=0.2, random_state=2018)
X_tr_speaker, X_te_speaker, y_tr_speaker, y_te_speaker = train_test_split(new_X_speaker, y_speaker, test_size=0.2, random_state=2018)
X_tr_rest, X_te_rest, y_tr_rest, y_te_rest = train_test_split(new_X_rest, y_rest, test_size=0.2, random_state=2018)
X_tr_hotels, X_te_hotels, y_tr_hotels, y_te_hotels = train_test_split(new_X_hotels, y_hotels, test_size=0.2, random_state=2018)

#We concateate the training set of five datasets
X_tr = X_tr_rest + X_tr_hotels + X_tr_comp + X_tr_lap + X_tr_router
X_val = X_te_rest + X_te_hotels + X_te_comp + X_te_lap + X_te_router
y_tr = np.concatenate((y_tr_rest, y_tr_hotels, y_tr_comp, y_tr_lap, y_tr_router), axis=0)
y_val = np.concatenate((y_te_rest, y_te_hotels, y_te_comp, y_te_lap, y_te_router), axis=0)

#The remaining one will be used as test set
X_te = X_tr_speaker + X_te_speaker
y_te = np.concatenate((y_tr_speaker, y_te_speaker), axis=0)

#If you are interested in testing on one of the datasets used for training, you only need to switch the variables

In [ ]:
#We take only a number of sentences that is a multple of 32 (we don't consider some sentences)
X_tr, X_val = X_tr[:(int(len(X_tr)/32))*32], X_val[:(int(len(X_val)/32))*32]
y_tr, y_val = y_tr[:(int(len(y_tr)/32))*32], y_val[:(int(len(y_val)/32))*32]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

#we add some sentences to the training set containing only padwords
for i in range(32 - len(X_te)%32):
  new_seq = []
  for j in range(max_len):
    new_seq.append("PADword")
  X_te.append(new_seq)

In [ ]:
for i in range(5):
  #we define the structure of the network and we print the network characteristics
  input_text = Input(shape=(max_len,), dtype=tf.string)
  embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
  x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
  x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
  x = add([x, x_rnn])  # residual connection to the first biLSTM
  out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
  model = Model(input_text, out)
  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  

  print("Run n° " + str(i+1))

  #fitting the model
  history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),batch_size=batch_size, epochs=10, verbose=1)
  
  #prediction on the test set
  test_pred = model.predict(np.array(X_te), verbose=1)
  pred_labels = pred2label(test_pred)
  test_labels = test2label(y_te)

  #print the classification report
  print(classification_report(test_labels,pred_labels[:len(new_X_speaker)]))

  #print the f1 of each sentence on a file for a future statistical analysis 
  print_f1_on_file("/content/drive/My Drive/Tesi_ABSA/speaker_elmo_full_f1_"+str(i+1)+".txt",test_labels,pred_labels[:len(new_X_speaker)])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


Run n° 1
Train on 8128 samples, validate on 2016 samples
Epoch 1/10
 192/8128 [..............................] - ETA: 2:02:51 - loss: 0.3143 - accuracy: 0.9079

KeyboardInterrupt: ignored